Data directories:

`/data/gersbachlab/lrb53/SczProveIt/mhc2/analysis/abcmodel/inputs/k562
/data/gersbachlab/lrb53/SczProveIt/mhc2/analysis/abcmodel/inputs/ipsc
/data/gersbachlab/lrb53/SczProveIt/mhc2/analysis/abcmodel/inputs/npc`


`/data/reddylab/Alex/collab/20220203_Lexi/data/ipsc.expect_cells.protospacer_dhs.w_pos_ctrls_dhs.txt`


In [115]:
!ls /data/gersbachlab/lrb53/SczProveIt/mhc2/analysis/abcmodel/inputs/npc

npc.atac.rep1.bam     npc.h3k27ac.rep2.bam  npc.hic.hg19.txt.gz
npc.atac.rep2.bam     npc.h3k27ac.rep3.bam
npc.h3k27ac.rep1.bam  npc.hicar.bedpe


In [4]:
!wc -l /data/reddylab/Alex/collab/20220203_Lexi/data/ipsc.expect_cells.protospacer_dhs.w_pos_ctrls_dhs.txt

12617 /data/reddylab/Alex/collab/20220203_Lexi/data/ipsc.expect_cells.protospacer_dhs.w_pos_ctrls_dhs.txt


In [5]:
!tail -n+2 /data/reddylab/Alex/collab/20220203_Lexi/data/ipsc.expect_cells.protospacer_dhs.w_pos_ctrls_dhs.txt | cut -f2 | sort -u | wc -l

598


In [23]:
import pandas as pd

DHS = pd.read_csv('/data/reddylab/Alex/collab/20220203_Lexi/data/ipsc.expect_cells.protospacer_dhs.w_pos_ctrls_dhs.txt', sep='\t')
DHS[['chr','start','end','name']] = DHS['dhs'].str.split(r':|-|_', expand=True)
DHS[['chr','start','end']].drop_duplicates().to_csv('/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/ipsc.expect_cells.protospacer_dhs.w_pos_ctrls_dhs.coordinates.bed', sep='\t', header=False, index=None)

In [63]:
%%bash
awk -F '\t' -vOFS='\t' '{$1=$1"\t"1; print }' /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/reference/hg38.genome \
> /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/reference/hg38.genome.bed

# iPSC

In [3]:
!ls /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/iPSC

bam		 esc.hicar.bedpe.gz  HiC
esc.hicar.bedpe  esc.hicar.v1.bedpe  ipsc.mean_tpms_bulk.txt


create index for the bam files

In [74]:
%%bash
mkdir -p /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/iPSC
module load samtools 
bam_files=$(/bin/ls /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/iPSC/*.bam)
for bam in ${bam_files};
do
samtools index -b ${bam} > ${bam/.bam/.bam.bai}
done

annotate the genes tpm file with genomic coordinates

In [19]:
!head /data/reddylab/Revathy/collabs/Maria/ABC/ABC-Enhancer-Gene-Prediction/reference/RefSeqCurated.170308.bed.CollapsedGeneBounds.bed

chr1	11873	14409	DDX11L1	0	+
chr1	14361	29370	WASH7P	0	-
chr1	34610	36081	FAM138A	0	-
chr1	34610	36081	FAM138F	0	-
chr1	69090	70008	OR4F5	0	+
chr1	134772	140566	LOC729737	0	-
chr1	323891	328581	LOC100132062	0	+
chr1	323891	328581	LOC100132287	0	+
chr1	562759	564389	LOC101928626	0	-
chr1	700244	714068	LOC100288069	0	-


In [10]:
import pandas as pd

gene_tpm = pd.read_csv('/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/iPSC/ipsc.mean_tpms_bulk.txt', sep='\t', names=['GeneName','tpm'])
gene_annotation = pd.read_csv('/data/reddylab/Revathy/collabs/Maria/ABC/ABC-Enhancer-Gene-Prediction/reference/RefSeqCurated.170308.bed.CollapsedGeneBounds.bed', sep='\t',
                             names=['chr','start','end','GeneName','number','strand'])
gene_bed = gene_tpm.merge(gene_annotation, on='GeneName', how='inner')
cols = ['start','end','number']
gene_bed[cols] = gene_bed[cols].apply(pd.to_numeric, errors='coerce')
gene_bed[['chr','start','end','GeneName','number','strand']].to_csv('/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/iPSC/ipsc.tpm_gene_list.coordinates.bed', sep='\t', index=False, header=None)

In [21]:
!head /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/iPSC/ipsc.tpm_gene_list.coordinates.bed

chr6	30539169.0	30559309.0	ABCF1	0.0	+
chr6	31654725.0	31671137.0	ABHD16A	0.0	-
chr6	32148744.0	32152099.0	AGER	0.0	-
chr6	32135982.0	32143916.0	AGPAT1	0.0	-
chr6	31582985.0	31584802.0	AIF1	0.0	+
chr6	31620186.0	31625987.0	APOM	0.0	+
chr12	103351451.0	103354294.0	ASCL1	0.0	+
chr6	30594618.0	30612448.0	ATAT1	0.0	+
chr6	32083044.0	32096017.0	ATF6B	0.0	-
chr6	31512227.0	31514625.0	ATP6V1G2	0.0	-


In [17]:
!ls /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/iPSC

bam		    esc.hicar.v1.bedpe	     ipsc.tpm_gene_list.coordinates.bed
esc.hicar.bedpe     HiC
esc.hicar.bedpe.gz  ipsc.mean_tpms_bulk.txt


In [5]:
%%bash
task=enhancer_activity
mkdir -p /data/reddylab/Revathy/collabs/Lexi/ABC_model/results
source /data/reddylab/software/miniconda3/bin/activate alex_py3
module load bedtools2
module load samtools
data_dir=/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/iPSC
abc_dir=/data/reddylab/Revathy/collabs/Maria/ABC/ABC-Enhancer-Gene-Prediction

mkdir -p /data/reddylab/Revathy/collabs/Lexi/ABC_model/results/${task}/iPSC/NoGeneExpr

python ${abc_dir}/src/run.neighborhoods.py \
--candidate_enhancer_regions /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/ipsc.expect_cells.protospacer_dhs.w_pos_ctrls_dhs.coordinates.bed \
--genes ${abc_dir}/reference/RefSeqCurated.170308.bed.CollapsedGeneBounds.bed \
--H3K27ac ${data_dir}/bam/ipsc.h3k27ac.rep1.bam,${data_dir}/bam/ipsc.h3k27ac.rep2.bam \
--DHS ${data_dir}/bam/ipsc.atac.rep1.bam,${data_dir}/bam/ipsc.atac.rep2.bam \
--chrom_sizes /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/reference/hg38.genome \
--cellType iPSC \
--outdir /data/reddylab/Revathy/collabs/Lexi/ABC_model/results/${task}/iPSC/NoGeneExpr

Namespace(ATAC='', DHS='/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/iPSC/bam/ipsc.atac.rep1.bam,/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/iPSC/bam/ipsc.atac.rep2.bam', H3K27ac='/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/iPSC/bam/ipsc.h3k27ac.rep1.bam,/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/iPSC/bam/ipsc.h3k27ac.rep2.bam', candidate_enhancer_regions='/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/ipsc.expect_cells.protospacer_dhs.w_pos_ctrls_dhs.coordinates.bed', cellType='iPSC', chrom_sizes='/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/reference/hg38.genome', default_accessibility_feature=None, enhancer_class_override=None, expression_table='', gene_name_annotations='symbol', genes='/data/reddylab/Revathy/collabs/Maria/ABC/ABC-Enhancer-Gene-Prediction/reference/RefSeqCurated.170308.bed.CollapsedGeneBounds.bed', genes_for_class_assignment=None, outdir='/data/reddylab/Revathy/collabs/Lexi/ABC_model/results/enhancer_activity/iPSC/NoGeneExpr', p

In [49]:
%%bash
awk -vOFS="\t" '{print $1,$2,$3,$4,$5,$6,$10,$8}' /data/gersbachlab/lrb53/SczProveIt/mhc2/analysis/abcmodel/inputs/ipsc/esc.hicar.bedpe \
| tail -n+2 \
| gzip -c \
> /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/iPSC/esc.hicar.bedpe.gz

In [84]:
%%bash
chrom=("chr19" "chr20" "chr21" "chr22" "chrX" "chrY")
for chr in "${chrom[@]}"
do
ln -s /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/iPSC/esc.hicar.bedpe.gz \
/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/iPSC/HiC/${chr}/${chr}.bedpe.gz
done

In [2]:
%%bash
source /data/reddylab/software/miniconda3/bin/activate alex_py3
task=ABC_score
mkdir -p /data/reddylab/Revathy/collabs/Lexi/ABC_model/results/${task}
abc_dir=/data/reddylab/Revathy/collabs/Maria/ABC/ABC-Enhancer-Gene-Prediction
enhancer_activity=/data/reddylab/Revathy/collabs/Lexi/ABC_model/results/enhancer_activity/iPSC/
hic=/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/iPSC/HiC

cd ${enhancer_activity}

python ${abc_dir}/src/predict.py \
--enhancers ${enhancer_activity}/EnhancerList.txt \
--genes ${enhancer_activity}/GeneList.txt \
--HiCdir ${hic} \
--hic_resolution 5000 \
--hic_type bedpe \
--scale_hic_using_powerlaw \
--threshold .02 \
--cellType iPSC \
--outdir /data/reddylab/Revathy/collabs/Lexi/ABC_model/results/${task}/iPSC/ \
--make_all_putative 


reading genes
reading enhancers
Making predictions for chromosome: chr3
Making putative predictions table...
Begin HiC
Loading HiC
0 0
HiC added to predictions table. Elapsed time: 0.15144658088684082
HiC Complete
Completed chromosome: chr3. Elapsed time: 0.4517557621002197 

Making predictions for chromosome: chr6
Making putative predictions table...
Begin HiC
Loading HiC
269 43
HiC added to predictions table. Elapsed time: 0.25264525413513184
HiC Complete
Completed chromosome: chr6. Elapsed time: 0.6411004066467285 

Making predictions for chromosome: chr2
Making putative predictions table...
Begin HiC
Loading HiC
0 0
HiC added to predictions table. Elapsed time: 0.1361403465270996
HiC Complete
Completed chromosome: chr2. Elapsed time: 0.29071784019470215 

Making predictions for chromosome: chr4
Making putative predictions table...
Begin HiC
Loading HiC
2 0
HiC added to predictions table. Elapsed time: 0.1494450569152832
HiC Complete
Completed chromosome: chr4. Elapsed time: 0.26790

# K562

In [87]:
%%bash
module load samtools 
mkdir -p /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/K562
cd /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/K562
cp /data/gersbachlab/lrb53/SczProveIt/mhc2/analysis/abcmodel/inputs/k562/*.bam .
bam_files=$(/bin/ls /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/K562/*.bam)
for bam in ${bam_files};
do
samtools index -b ${bam} > ${bam/.bam/.bam.bai}
done

In [3]:
!wc -l /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/K562/k562.mean_tpms_bulk.txt

156 /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/K562/k562.mean_tpms_bulk.txt


In [4]:
import pandas as pd

gene_tpm = pd.read_csv('/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/K562/k562.mean_tpms_bulk.txt', sep='\t', names=['GeneName','tpm'])
gene_annotation = pd.read_csv('/data/reddylab/Revathy/collabs/Maria/ABC/ABC-Enhancer-Gene-Prediction/reference/RefSeqCurated.170308.bed.CollapsedGeneBounds.bed', sep='\t',
                             names=['chr','start','end','GeneName','number','strand'])
gene_bed = gene_tpm.merge(gene_annotation, on='GeneName', how='inner')
cols = ['start','end','number']
gene_bed[cols] = gene_bed[cols].apply(pd.to_numeric, errors='coerce')
gene_bed[['chr','start','end','GeneName','number','strand']].to_csv('/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/K562/K562.tpm_gene_list.coordinates.bed', sep='\t', index=False, header=None)


In [6]:
!wc -l /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/K562/K562.tpm_gene_list.coordinates.bed

145 /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/K562/K562.tpm_gene_list.coordinates.bed


In [7]:
%%bash
task=enhancer_activity
mkdir -p /data/reddylab/Revathy/collabs/Lexi/ABC_model/results
source /data/reddylab/software/miniconda3/bin/activate alex_py3
module load bedtools2
module load samtools
data_dir=/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/K562
abc_dir=/data/reddylab/Revathy/collabs/Maria/ABC/ABC-Enhancer-Gene-Prediction

mkdir -p /data/reddylab/Revathy/collabs/Lexi/ABC_model/results/${task}/K562/NoGeneExpr

python ${abc_dir}/src/run.neighborhoods.py \
--candidate_enhancer_regions /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/ipsc.expect_cells.protospacer_dhs.w_pos_ctrls_dhs.coordinates.bed \
--genes ${abc_dir}/reference/RefSeqCurated.170308.bed.CollapsedGeneBounds.bed \
--H3K27ac ${data_dir}/k562.h3k27ac.rep1.bam,${data_dir}/k562.h3k27ac.rep2.bam,${data_dir}/k562.h3k27ac.rep3.bam \
--DHS ${data_dir}/k562.atac.rep1.bam,${data_dir}/k562.atac.rep2.bam,${data_dir}/k562.atac.rep3.bam \
--chrom_sizes /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/reference/hg38.genome \
--cellType K562 \
--outdir /data/reddylab/Revathy/collabs/Lexi/ABC_model/results/${task}/K562/NoGeneExpr

Namespace(ATAC='', DHS='/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/K562/k562.atac.rep1.bam,/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/K562/k562.atac.rep2.bam,/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/K562/k562.atac.rep3.bam', H3K27ac='/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/K562/k562.h3k27ac.rep1.bam,/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/K562/k562.h3k27ac.rep2.bam,/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/K562/k562.h3k27ac.rep3.bam', candidate_enhancer_regions='/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/ipsc.expect_cells.protospacer_dhs.w_pos_ctrls_dhs.coordinates.bed', cellType='K562', chrom_sizes='/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/reference/hg38.genome', default_accessibility_feature=None, enhancer_class_override=None, expression_table='', gene_name_annotations='symbol', genes='/data/reddylab/Revathy/collabs/Maria/ABC/ABC-Enhancer-Gene-Prediction/reference/RefSeqCurated.170308.bed.CollapsedGeneBounds.b

In [52]:
%%bash
zcat /data/gersbachlab/lrb53/SczProveIt/mhc2/analysis/abcmodel/inputs/k562/*.gz | cut -f1,2,3,4,5,6,7,12 | tail -n+3 \
| gzip -c \
> /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/K562/k562.hic.bedpe.gz

In [10]:
%%bash
chrom=("chr1" "chr19" "chr20" "chr21" "chr22" "chrX" "chrY" "chr2" "chr3" "chr4" "chr5" "chr6" "chr7" "chr8" "chr9" "chr10" "chr11" "chr12" "chr13" "chr14" "chr15" "chr16" "chr17" "chr18")
for chr in "${chrom[@]}"
do
mkdir -p /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/K562/HiC/${chr}
ln -s /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/K562/k562.hic.bedpe.gz \
/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/K562/HiC/${chr}/${chr}.bedpe.gz
done

In [8]:
%%bash
source /data/reddylab/software/miniconda3/bin/activate alex_py3
task=ABC_score
mkdir -p /data/reddylab/Revathy/collabs/Lexi/ABC_model/results/${task}
abc_dir=/data/reddylab/Revathy/collabs/Maria/ABC/ABC-Enhancer-Gene-Prediction
enhancer_activity=/data/reddylab/Revathy/collabs/Lexi/ABC_model/results/enhancer_activity/K562/NoGeneExpr
hic=/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/K562/HiC

cd ${enhancer_activity}

python ${abc_dir}/src/predict.py \
--enhancers ${enhancer_activity}/EnhancerList.txt \
--genes ${enhancer_activity}/GeneList.txt \
--HiCdir ${hic} \
--hic_resolution 5000 \
--hic_type bedpe \
--scale_hic_using_powerlaw \
--threshold .02 \
--cellType K562 \
--outdir /data/reddylab/Revathy/collabs/Lexi/ABC_model/results/${task}/K562/NoGeneExpr \
--make_all_putative 


reading genes
reading enhancers
Making predictions for chromosome: chr12
Making putative predictions table...
Begin HiC
Loading HiC
0 67
HiC added to predictions table. Elapsed time: 0.15626096725463867
HiC Complete
Completed chromosome: chr12. Elapsed time: 0.5024785995483398 

Making predictions for chromosome: chr6
Making putative predictions table...
Begin HiC
Loading HiC
72 94
HiC added to predictions table. Elapsed time: 0.3287363052368164
HiC Complete
Completed chromosome: chr6. Elapsed time: 0.9499437808990479 

Making predictions for chromosome: chr2
Making putative predictions table...
Begin HiC
Loading HiC
0 79
HiC added to predictions table. Elapsed time: 0.14385390281677246
HiC Complete
Completed chromosome: chr2. Elapsed time: 0.23351716995239258 

Making predictions for chromosome: chr11
Making putative predictions table...
Begin HiC
Loading HiC
0 108
HiC added to predictions table. Elapsed time: 0.14253640174865723
HiC Complete
Completed chromosome: chr11. Elapsed time:

# NPC

In [2]:
%%bash
mkdir -p /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/NPC
mkdir -p /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/NPC/bam
cp /data/gersbachlab/lrb53/SczProveIt/mhc2/analysis/abcmodel/inputs/npc/*.bam /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/NPC/bam
module load samtools 
bam_files=$(/bin/ls /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/NPC/bam/*.bam)
for bam in ${bam_files};
do
samtools index -b ${bam} > ${bam/.bam/.bam.bai}
done

In [1]:
!wc -l /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/NPC/npc.mean_tpms_bulk.txt

150 /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/NPC/npc.mean_tpms_bulk.txt


In [2]:
import pandas as pd

gene_tpm = pd.read_csv('/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/NPC/npc.mean_tpms_bulk.txt', sep='\t', names=['GeneName','tpm'])
gene_annotation = pd.read_csv('/data/reddylab/Revathy/collabs/Maria/ABC/ABC-Enhancer-Gene-Prediction/reference/RefSeqCurated.170308.bed.CollapsedGeneBounds.bed', sep='\t',
                             names=['chr','start','end','GeneName','number','strand'])
gene_bed = gene_tpm.merge(gene_annotation, on='GeneName', how='inner')
cols = ['start','end','number']
gene_bed[cols] = gene_bed[cols].apply(pd.to_numeric, errors='coerce')
gene_bed[['chr','start','end','GeneName','number','strand']].to_csv('/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/NPC/NPC.tpm_gene_list.coordinates.bed', sep='\t', index=False, header=None)


In [9]:
%%bash
task=enhancer_activity
source /data/reddylab/software/miniconda3/bin/activate alex_py3
module load bedtools2
module load samtools
data_dir=/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/NPC
abc_dir=/data/reddylab/Revathy/collabs/Maria/ABC/ABC-Enhancer-Gene-Prediction

mkdir -p /data/reddylab/Revathy/collabs/Lexi/ABC_model/results/${task}/NPC/NoGeneExpr

python ${abc_dir}/src/run.neighborhoods.py \
--candidate_enhancer_regions /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/ipsc.expect_cells.protospacer_dhs.w_pos_ctrls_dhs.coordinates.bed \
--genes ${abc_dir}/reference/RefSeqCurated.170308.bed.CollapsedGeneBounds.bed \
--H3K27ac ${data_dir}/bam/npc.h3k27ac.rep1.bam,${data_dir}/bam/npc.h3k27ac.rep2.bam,${data_dir}/bam/npc.h3k27ac.rep3.bam \
--DHS ${data_dir}/bam/npc.atac.rep1.bam,${data_dir}/bam/npc.atac.rep2.bam \
--chrom_sizes /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/reference/hg38.genome \
--cellType NPC \
--outdir /data/reddylab/Revathy/collabs/Lexi/ABC_model/results/${task}/NPC/NoGeneExpr

Namespace(ATAC='', DHS='/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/NPC/bam/npc.atac.rep1.bam,/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/NPC/bam/npc.atac.rep2.bam', H3K27ac='/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/NPC/bam/npc.h3k27ac.rep1.bam,/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/NPC/bam/npc.h3k27ac.rep2.bam,/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/NPC/bam/npc.h3k27ac.rep3.bam', candidate_enhancer_regions='/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/ipsc.expect_cells.protospacer_dhs.w_pos_ctrls_dhs.coordinates.bed', cellType='NPC', chrom_sizes='/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/reference/hg38.genome', default_accessibility_feature=None, enhancer_class_override=None, expression_table='', gene_name_annotations='symbol', genes='/data/reddylab/Revathy/collabs/Maria/ABC/ABC-Enhancer-Gene-Prediction/reference/RefSeqCurated.170308.bed.CollapsedGeneBounds.bed', genes_for_class_assignment=None, outdir='/data/reddylab/Revat

In [6]:
%%bash
awk -vOFS="\t" '{print $1,$2,$3,$4,$5,$6,$10,$8}' /data/gersbachlab/lrb53/SczProveIt/mhc2/analysis/abcmodel/inputs/npc/npc.hicar.bedpe \
| tail -n+2 \
| gzip -c \
> /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/NPC/npc.hicar.bedpe.gz

In [9]:
%%bash
chrom=("chr1" "chr19" "chr20" "chr21" "chr22" "chrX" "chrY" "chr2" "chr3" "chr4" "chr5" "chr6" "chr7" "chr8" "chr9" "chr10" "chr11" "chr12" "chr13" "chr14" "chr15" "chr16" "chr17" "chr18")
for chr in "${chrom[@]}"
do
mkdir -p /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/NPC/HiC/${chr}
ln -s /data/reddylab/Revathy/collabs/Lexi/ABC_model/data/NPC/npc.hicar.bedpe.gz \
/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/NPC/HiC/${chr}/${chr}.bedpe.gz
done

In [1]:
%%bash
source /data/reddylab/software/miniconda3/bin/activate alex_py3
task=ABC_score
abc_dir=/data/reddylab/Revathy/collabs/Maria/ABC/ABC-Enhancer-Gene-Prediction
enhancer_activity=/data/reddylab/Revathy/collabs/Lexi/ABC_model/results/enhancer_activity/NPC/
hic=/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/NPC/HiC

cd ${enhancer_activity}

python ${abc_dir}/src/predict.py \
--enhancers ${enhancer_activity}/EnhancerList.txt \
--genes ${enhancer_activity}/GeneList.txt \
--HiCdir ${hic} \
--hic_resolution 5000 \
--hic_type bedpe \
--scale_hic_using_powerlaw \
--threshold .02 \
--cellType NPC \
--outdir /data/reddylab/Revathy/collabs/Lexi/ABC_model/results/${task}/NPC/ \
--make_all_putative 


reading genes
reading enhancers
Making predictions for chromosome: chr11
Making putative predictions table...
Begin HiC
Loading HiC
0 0
HiC added to predictions table. Elapsed time: 0.34586334228515625
HiC Complete
Completed chromosome: chr11. Elapsed time: 1.0910766124725342 

Making predictions for chromosome: chr3
Making putative predictions table...
Begin HiC
Loading HiC
0 0
HiC added to predictions table. Elapsed time: 0.1614236831665039
HiC Complete
Completed chromosome: chr3. Elapsed time: 0.31017470359802246 

Making predictions for chromosome: chr2
Making putative predictions table...
Begin HiC
Loading HiC
0 0
HiC added to predictions table. Elapsed time: 0.20608758926391602
HiC Complete
Completed chromosome: chr2. Elapsed time: 0.36472272872924805 

Making predictions for chromosome: chr12
Making putative predictions table...
Begin HiC
Loading HiC
0 0
HiC added to predictions table. Elapsed time: 0.15871167182922363
HiC Complete
Completed chromosome: chr12. Elapsed time: 0.30

In [ ]:
import pandas as pd

k562_hits = pd.read_csv('/data/reddylab/Revathy/collabs/Lexi/ABC_model/data/v14mhc.dhs.int.k562.screenccresv4.hg38.txt', sep='\t',
                       names=['dhs_chr','dhs_start','dhs_end','cre_type','ccre_chr','ccre_start','ccre_end','ccre_id','ccre_score',
                              'ccre_strand','ccre_thickStart','ccre_thickEnd','ccre_color','ccre_type','ccre_analysis'])

k562_hits['dhs_chr_start_end'] = k562_hits['dhs_chr']+'_'+k562_hits['dhs_start'].astype(str)+'_'+k562_hits['dhs_end'].astype(str)
k562_hits.groupby(['cre_type','ccre_type']).size().sort_values(ascending=False).reset_index(name='count') 